In [99]:
import pandas as pd
from pptx import Presentation
from pptx.util import Inches
from pptx.dml.color import RGBColor
from pptx.enum.shapes import MSO_SHAPE
from pptx.oxml import parse_xml
from pptx.oxml.ns import nsdecls
from pptx.util import Pt
import sys
from pptx.enum.shapes import MSO_CONNECTOR
import numpy as np

In [100]:
def build_hierarchy(df, employee_id, level=1):
    """
    Recursively finds all employees under a given employee ID and assigns a level to them.
    
    Args:
        df (pd.DataFrame): The dataframe containing 'Employee ID' and 'Supervisor ID'.
        employee_id (int or str): The employee ID from which to start building the hierarchy.
        level (int): The current hierarchical level (default is 1 for the given employee_id).

    Returns:
        pd.DataFrame: A dataframe with all original columns plus 'Level', containing only employees under the given employee ID.
    """
    hierarchy = []
    #root = df[df['Employee ID'] == employee_id]
    #root = root.reset_index(drop=True)
    
    def recurse(emp_id, lvl):
        
        subordinates = df[df['Supervisor ID'] == emp_id]
        for _, row in subordinates.iterrows():
            row_dict = row.to_dict()
            row_dict['Level'] = lvl
            hierarchy.append(row_dict)
            recurse(row['Employee ID'], lvl + 1)
    
    # Check if the given employee exists in the dataset
    if employee_id not in df['Employee ID'].values:
        return pd.DataFrame(columns=df.columns.tolist() + ['Level'])
    
    # Start with the given employee
    root_employee = df[df['Employee ID'] == employee_id].iloc[0].to_dict()
    root_employee['Level'] = level
    hierarchy.append(root_employee)
    
    recurse(employee_id, level + 1)
    
    return pd.DataFrame(hierarchy)


def group(result_df):
    grouped = result_df.groupby(["Supervisor ID", "Level"])

    two_level_heirarchy_dict = {}

    for _, group in grouped:
        
        if group.iloc[0]['Level'] != 1: # if not root
            
            two_level_heirarchy = []
            supervisor_id = int(group.iloc[0]["Supervisor ID"])
            
            for _, row in group.iterrows():
                two_level_heirarchy.append(row.to_dict())
            #two_level_heirarchy.append(False) # To indicate if the structure has been plotted
                
                
            two_level_heirarchy_dict[int(supervisor_id)] = two_level_heirarchy
            
    return two_level_heirarchy_dict

In [101]:

data = {
    'Employee ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Supervisor ID': [0, 1, 1, 1, 2, 2, 5, 5, 7, 5],
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve', 'Frank', 'Grace', 'Jack', 'Tim', 'Sarah'],
    'Department': ['HR', 'IT', 'IT', 'Finance', 'Finance', 'IT', 'Finance', 'Cyber', 'Cyber', 'HR']
}

'''
1 -> (2, 3, 4)
2 -> (5, 6)
5 -> (7, 8, 10)
7 -> (9)
'''

df = pd.DataFrame(data)
result_df = build_hierarchy(df, 1)
branch = group(result_df)
root = df[df["Employee ID"] == 1].iloc[0].to_dict() 

In [102]:
root

{'Employee ID': 1, 'Supervisor ID': 0, 'Name': 'Alice', 'Department': 'HR'}

In [150]:
def add_rectangle_root(slide, left, top, width, height, root):
    """Adds a rectangle with text at the specified position."""
    
    '''
    left -	X-coordinate (distance from the left side of the slide) 
    top	 -  Y-coordinate (distance from the top of the slide) 
    width -	Width of the shape 
    height -  Height of the shape 
    '''
    
    text = f"{root['Department']}\n{root['Name']} ({root['Employee ID']})"
    
    shape = slide.shapes.add_shape(MSO_SHAPE.RECTANGLE, left, top, width, height)
    shape.fill.solid()
    shape.fill.fore_color.rgb = RGBColor(10, 80, 110)  # Dark blue
    shape.text_frame.text = text
    shape.text_frame.paragraphs[0].font.size = Pt(16)
    shape.text_frame.paragraphs[1].font.size = Pt(16)
    shape.text_frame.paragraphs[0].font.bold = True
    shape.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255)  # White text
    #self.coordinates[self.root['Employee ID']] = shape
    
    return {"target": (root['Employee ID'], shape)}
        

In [204]:
def deep_search(slide, target, branch, layer=0):
    target_id = target['target'][0]
    target_shape = target['target'][1]
    
    offset_x = target_shape.left + Inches(0.1)
    x_offset_h = target_shape.width / 4
    
    if layer == 0:
        gap = 0.68 * target_shape.height  # Fixed vertical gap between shapes
    else:
        gap = 1 * target_shape.height  
        
    if target_id in branch:
        subordinates = branch[target_id]
        total_height_used = 0  # Tracks cumulative height used by this branch
        
        for i, subordinate in enumerate(subordinates):
            
            # Calculate y-position based on previous subordinates' heights
            if layer == 0:
                y_pos = target_shape.top + (1 + 0.68 * (i + total_height_used)) * target_shape.height 
            else:
                y_pos = target_shape.top + (1 + 1 * (i + total_height_used)) * target_shape.height
            
            # Draw the subordinate box
            text = f"{subordinate['Department']}\n{subordinate['Name']} ({subordinate['Employee ID']})"
            
            # Vertical line (from parent to horizontal connector)
            slide.shapes.add_connector(
                MSO_CONNECTOR.STRAIGHT, 
                int(offset_x), int(y_pos), 
                int(offset_x), int(y_pos + gap)
            )
            # Horizontal line (connecting to the box)
            slide.shapes.add_connector(
                MSO_CONNECTOR.STRAIGHT, 
                int(offset_x), int(y_pos + gap), 
                int(offset_x + x_offset_h), int(y_pos + gap)
            )
            # Subordinate rectangle
            shape = slide.shapes.add_shape(
                MSO_SHAPE.RECTANGLE, 
                int(offset_x + x_offset_h), 
                int(y_pos + gap / 2 + Inches(0.05)), 
                Inches(0.8), Inches(0.4)
            )
            shape.text_frame.text = text
            shape.text_frame.paragraphs[0].font.size = Pt(10)
            shape.text_frame.paragraphs[1].font.size = Pt(10)
            shape.text_frame.paragraphs[0].font.bold = True
            shape.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255)
            
            # Recursively process subordinates (if any)
            new_target_id = subordinate['Employee ID']
            if new_target_id in branch:
                # Get the height consumed by this subordinate's subtree
                subtree_height = deep_search(slide, {"target": (new_target_id, shape)}, branch, layer=1)
                total_height_used += subtree_height
            else:
                # If no further subordinates, count this as 1 unit of height
                total_height_used += 1
        
        return len(subordinates) + total_height_used  # Total shapes in this branch
    
    return 0  # No subordinates found

In [215]:
def deep_search(slide, target, branch, layer=0):
    target_id = target['target'][0]
    target_shape = target['target'][1]
    
    offset_x = target_shape.left + Inches(0.1)
    x_offset_h = target_shape.width / 4
    
    if layer == 0:
        gap = 0.68 * target_shape.height  # Fixed vertical gap between shapes
    else:
        gap = 1 * target_shape.height  
        
    if target_id in branch:
        subordinates = branch[target_id]
        total_height_used = 0  # Tracks cumulative height used by this branch
        node_positions = []  # Store all node positions in this level
        
        for i, subordinate in enumerate(subordinates):
            # Calculate y-position based on previous subordinates' heights
            if layer == 0:
                y_pos = target_shape.top + (1 + 0.68 * (i + total_height_used)) * target_shape.height 
            else:
                y_pos = target_shape.top + (1 + 1 * (i + total_height_used)) * target_shape.height
            
            # Draw the subordinate box
            text = f"{subordinate['Department']}\n{subordinate['Name']} ({subordinate['Employee ID']})"
            
            # Store node position for main vertical line
            node_positions.append(y_pos + gap/2)  # Middle of the horizontal connector
            
            # Horizontal line (connecting to the box)
            slide.shapes.add_connector(
                MSO_CONNECTOR.STRAIGHT, 
                int(offset_x), int(y_pos + gap), 
                int(offset_x + x_offset_h), int(y_pos + gap)
            )
            
            # Subordinate rectangle
            shape = slide.shapes.add_shape(
                MSO_SHAPE.RECTANGLE, 
                int(offset_x + x_offset_h), 
                int(y_pos + gap / 2 + Inches(0.05)), 
                Inches(0.8), Inches(0.4)
            )
            shape.text_frame.text = text
            shape.text_frame.paragraphs[0].font.size = Pt(10)
            shape.text_frame.paragraphs[1].font.size = Pt(10)
            shape.text_frame.paragraphs[0].font.bold = True
            shape.text_frame.paragraphs[0].font.color.rgb = RGBColor(255, 255, 255)
            
            # Recursively process subordinates (if any)
            new_target_id = subordinate['Employee ID']
            if new_target_id in branch:
                subtree_height = deep_search(slide, {"target": (new_target_id, shape)}, branch, layer=1)
                total_height_used += subtree_height
            else:
                total_height_used += 1
        
        # Draw single vertical line for this level
        if node_positions:
            min_y = min(node_positions) - gap/2
            max_y = max(node_positions) + gap/2
            slide.shapes.add_connector(
                MSO_CONNECTOR.STRAIGHT,
                int(offset_x), int(min_y),
                int(offset_x), int(max_y)
            )
            
            # Add small horizontal connectors from main vertical line to each node
            for y in node_positions:
                slide.shapes.add_connector(
                    MSO_CONNECTOR.STRAIGHT,
                    int(offset_x), int(y),
                    int(offset_x), int(y)  # Zero-length connector to create connection point
                )
        
        return len(subordinates) + total_height_used  # Total shapes in this branch
    
    return 0  # No subordinates found

In [216]:
prs = Presentation()
slide = prs.slides.add_slide(prs.slide_layouts[6])  

# Slide width
slide_width = prs.slide_width

# Root shape dimensions
shape_width = Inches(1)
shape_height = Inches(0.6)

left = (slide_width - shape_width) / 2  # Center horizontally
top = Inches(1)  # Leave space for title

target = add_rectangle_root(slide, left, top, shape_width, shape_height, root)

deep_search(slide, target, branch)



prs.save("deep_search.pptx")
print("Presentation saved as 'deep_search.pptx'")

Presentation saved as 'deep_search.pptx'
